In [ ]:
import geowombat as gw
import geopandas as gpd

polys = department

with gw.open('/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif') as src:
    print(src)
    clipped = src.gw.clip_by_polygon(
        polys,
        mask_data=True,
        expand_by=1
    )
    print(clipped)

In [ ]:
with gw.open('/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif') as src:
    df = src.gw.extract(department)

In [ ]:
for ruta_tif in tif_file_path:
    # Abrir el archivo TIF
    with gw.open(ruta_tif) as src:
        # Procesar el archivo TIF
        df = src.gw.extract(department)
        # Hacer algo con el resultado, por ejemplo, imprimirlo
        print(df)

In [ ]:
import geowombat as gw

with gw.config.update(sensor='bgr'):
    with gw.open('/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif') as src:
        df = src.gw.extract(
            department,
            band_names=src.band.values.tolist()
        )

In [ ]:
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats

# Definir el tamaño del lote (chunk size)
chunk_size = 1000  # Ajusta el tamaño del chunk según tus necesidades

# Dividir el shapefile en lotes
chunks = [department[i:i+chunk_size] for i in range(0, len(department), chunk_size)]

# Lista para almacenar los resultados
raster_data = []

# Iterar sobre cada chunk
for chunk in chunks:
    for raster_path in tif_file_path:
        # Procesar el chunk actual del shapefile y calcular estadísticas zonales para el archivo raster actual
        stats = zonal_stats(chunk, raster_path, stats="count sum", categorical=True, all_touched=True)
        
        # Convertir las estadísticas en un DataFrame
        stats_df = pd.DataFrame(stats)
        
        # Concatenar la geometría del chunk y las estadísticas del raster en un DataFrame
        df = pd.concat([chunk.reset_index(drop=True), stats_df], axis=1)
        
        # Agregar el DataFrame resultante a la lista
        raster_data.append(df)

# Concatenar todos los DataFrames en uno solo
final_result = pd.concat(raster_data, ignore_index=True)